In [ ]:
import os
import pandas as pd
import requests as req
import os
import dotenv
from dotenv import load_dotenv
import zipfile
import matplotlib.pyplot as plt
import numpy as np

pd.options.display.float_format = '{:.4f}'.format

In [ ]:
#download daily data from 2000 to 2020
for ano in range(2000, 2021):
    ano = str(ano) 

    url_dados_diarios = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/inf_diario_fi_{ano}.zip'

    download = req.get(url_dados_diarios)

    with open(f'inf_diario_fi_{ano}.zip', 'wb') as arquivo_cvm:
        arquivo_cvm.write(download.content)

    arquivo_zip = zipfile.ZipFile(f'inf_diario_fi_{ano}.zip')
    arquivo_zip.extractall(f'inf_diario_fi_{ano}')

    dados_fundos_2000_2020 = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep = ';', encoding= 'ISO-8859-1')

dados_fundos_2000_2020

In [ ]:
#download daily data from 2021 to date (due to a change in the pattern)

for ano in range(2021, 2025):
    for mes in range(1, 13):
        
        ano_str = str(ano)
        mes_str = str(mes).zfill(2)  

        url_nova = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{ano_str}{mes_str}.zip'

        download = req.get(url_nova)

        with open(f'inf_diario_fi_{ano_str}{mes_str}.zip', 'wb') as arquivo_cvm:
            arquivo_cvm.write(download.content)

        with zipfile.ZipFile(f'inf_diario_fi_{ano_str}{mes_str}.zip') as arquivo_zip:
            dados_diarios_2020_2024 = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep=';', encoding='ISO-8859-1')

In [ ]:
#merging the daily data dataframes

diretorio_arquivos = 'your_path'


dataframes_2000_2024 = []

for ano in range(2000, 2025):
    for mes in range(1, 13):
        ano_str = str(ano)  
        mes_str = str(mes).zfill(2)

        if ano < 2005:
            caminho_arquivo_zip = os.path.join(diretorio_arquivos, f'inf_diario_fi_{ano}.zip')
            nome_arquivo_csv = f'inf_diario_fi_{ano}.csv'
        elif ano < 2021:
            caminho_arquivo_zip = os.path.join(diretorio_arquivos, f'inf_diario_fi_{ano}.zip')
            nome_arquivo_csv = f'inf_diario_fi_{ano_str}{mes_str}.csv'
        else:
            caminho_arquivo_zip = os.path.join(diretorio_arquivos, f'inf_diario_fi_{ano_str}{mes_str}.zip')
            nome_arquivo_csv = f'inf_diario_fi_{ano_str}{mes_str}.csv'

        if os.path.exists(caminho_arquivo_zip):
            with zipfile.ZipFile(caminho_arquivo_zip) as arquivo_zip:
                if nome_arquivo_csv in arquivo_zip.namelist():
                    with arquivo_zip.open(nome_arquivo_csv) as arquivo_csv:
                        dados_fundos = pd.read_csv(arquivo_csv, sep=';', encoding='ISO-8859-1')
                        dataframes_2000_2024.append(dados_fundos)
                else:
                    print(f'O arquivo CSV {nome_arquivo_csv} não foi encontrado no ZIP para o ano de {ano_str} e mês {mes_str}.')
        else:
            print(f'O arquivo ZIP para o ano de {ano_str} e mês {mes_str} não foi encontrado.')

dados_diarios = pd.concat(dataframes_2000_2024, ignore_index=True)

dados_diarios = dados_diarios[['CNPJ_FUNDO', 'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'CAPTC_DIA', 'RESG_DIA', 'NR_COTST']]

dados_diarios

In [ ]:
#download registration information from the Hedge Funds:

dados_cadastrais = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv',
                               sep= ';', encoding = 'ISO8859-1')

dados_cadastrais = dados_cadastrais.drop_duplicates()

In [ ]:
#merge it all together:

dados_diarios_completos = pd.merge(dados_diarios, dados_cadastrais, how= 'left',
                                      left_on= ['CNPJ_FUNDO'], right_on= ['CNPJ_FUNDO'])